<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/API_processing_nolabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install requests

# EXPERIMENT API to get the prefix easily





In [ ]:
import requests

def fetch_data(query):
    # Define the URL for the API request
    url = "https://lov.linkeddata.es/dataset/lov/api/v2/term/search"

    # Define the parameters for the request using the query variable
    params = {
        'q': query,
        'type': 'property',
        'page_size'	: 1
    }

    # Make the GET request to the API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

lala = fetch_data('citizenship')
lala

{'total_results': 5,
 'page': 1,
 'page_size': '1',
 'queryString': 'citizenship',
 'filters': {'type': 'property'},
 'aggregations': {'types': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'property', 'doc_count': 5}]},
  'vocabs': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'cwrc', 'doc_count': 2},
    {'key': 'being', 'doc_count': 1},
    {'key': 'dbpedia', 'doc_count': 1},
    {'key': 'owl', 'doc_count': 1},
    {'key': 'person', 'doc_count': 1}]},
  'tags': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'People', 'doc_count': 3},
    {'key': 'General & Upper', 'doc_count': 1},
    {'key': 'Multimedia', 'doc_count': 1}]}},
 'results': [{'prefixedName': ['dbpedia-owl:citizenship'],
   'vocabulary.prefix': ['dbpedia-owl'],
   'uri': ['http://dbpedia.org/ontology/citizenship'],
   'metrics.occurrencesInDatasets': [315],
   'metrics.reusedByDatasets': [2],
   'typ

In [ ]:
lala = fetch_data('winner')
lala

{'total_results': 13,
 'page': 1,
 'page_size': '1',
 'queryString': 'winner',
 'filters': {'type': 'property'},
 'aggregations': {'types': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'property', 'doc_count': 13}]},
  'vocabs': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'dbpedia', 'doc_count': 11},
    {'key': 'owl', 'doc_count': 11},
    {'key': 'game', 'doc_count': 1},
    {'key': 'schema', 'doc_count': 1}]},
  'tags': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'General & Upper', 'doc_count': 12},
    {'key': 'Games', 'doc_count': 1}]}},
 'results': [{'prefixedName': ['schema:winner'],
   'vocabulary.prefix': ['schema'],
   'uri': ['http://schema.org/winner'],
   'metrics.occurrencesInDatasets': [0],
   'metrics.reusedByDatasets': [0],
   'type': 'property',
   'score': 2.7106304,
   'highlight': {'http://www.w3.org/2000/01/rdf-schema#comment': ['A sub pr

In [ ]:
if 'results' in lala and len(lala['results']) > 0:
    result = lala['results'][0]

result

{'prefixedName': ['schema:winner'],
 'vocabulary.prefix': ['schema'],
 'uri': ['http://schema.org/winner'],
 'metrics.occurrencesInDatasets': [0],
 'metrics.reusedByDatasets': [0],
 'type': 'property',
 'score': 2.7106304,
 'highlight': {'http://www.w3.org/2000/01/rdf-schema#comment': ['A sub property of participant. The <b>winner</b> of the action.'],
  'http://www.w3.org/2000/01/rdf-schema#label': ['<b>winner</b>'],
  'localName.ngram': ['<b>winner</b>']}}

In [ ]:
RDFprefixes_names = {}

In [ ]:
import requests

def extract_and_strip(data):
    # Extract the relevant values
    prefixed_name = data['results'][0]['prefixedName'][0].strip()
    vocabulary_prefix = data['results'][0]['vocabulary.prefix'][0].strip()
    uri = data['results'][0]['uri'][0].strip()

    return {
        'prefixedName': prefixed_name,
        'vocabulary.prefix': vocabulary_prefix,
        'uri': uri
    }

def fetch_data(query):
    # Define the URL for the API request
    url = "https://lov.linkeddata.es/dataset/lov/api/v2/term/search"

    # Define the parameters for the request using the query variable
    params = {
        'q': query,
        'type': 'property',
        'page_size': 1
    }

    # Make the GET request to the API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Extract and strip the required information
        extracted_data = extract_and_strip(data)
        return extracted_data
    else:
        print(f"Error: {response.status_code}")
        return None

# Example usage:
query = 'citizenship'
result = fetch_data(query)
print(result)


{'prefixedName': 'dbpedia-owl:citizenship', 'vocabulary.prefix': 'dbpedia-owl', 'uri': 'http://dbpedia.org/ontology/citizenship'}


In [ ]:
if 'vocabulary.prefix' in result:
    vocabulary_prefix = result['vocabulary.prefix']

vocabulary_prefix

'dbpedia-owl'

#  Processing WN_test.tsv without label

In [ ]:
def process_tsv(file_path, max_tokens=1000):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sentences = []
    current_sentence = []
    sentence_enders = ".!?"

    for line in lines:
        token = line.strip()
        if token:
            current_sentence.append(token)

            if token in sentence_enders:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []

    if current_sentence:
        sentences.append(current_sentence)

    return sentences

file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/WN_test_nolabel.tsv'
sentences = process_tsv(file_path)

In [ ]:
titles = []
title_count = 0

for sentence in sentences:
    if sentence[-1] not in ".;?!:":
        titles.append(sentence)
        title_count += 1

print(f"Number of sentences appended to titles: {title_count}")

#dovrebbe essere il numero degli articoli

Number of sentences appended to titles: 290


In [ ]:
def create_dataset_NOlabel(sentences):
    dataset_list = []
    counterId = 0
    titles = []

    for sentence in sentences:
        if sentence[-1] not in ".;?!:":
            titles.append(sentence)

    for i, sentence in enumerate(sentences):
        if sentence[-1] not in ".;?!:":
            articles_dict = {}
            articles_dict['id'] = counterId
            articles_dict['title'] = sentence
            articles_dict['article'] = ''

            for j in range(i + 1, len(sentences)):
                if sentences[j] in titles:
                    break
                articles_dict['article'] += ' '.join(sentences[j]) + ' '

            dataset_list.append(articles_dict)
            counterId += 1

    return dataset_list

In [ ]:
dataset_list = create_dataset_NOlabel(sentences)

In [ ]:
dataset_list[1]

{'id': 1,
 'title': ['Rivolta',
  'del',
  'pane',
  'nel',
  'Magreb',
  ',',
  '50',
  'morti',
  'in',
  'Tunisia'],
 'article': '50 morti in Tunisia , scontri in Algeria e tensioni in Marocco , aumenti di prezzo per grano , olio , zucchero , oltre che del petrolio . La sommossa ricorda quella dell\' assalto al forno delle grucce , di manzoniana memoria . È di nuovo la " rivolta del pane " , in tutto il Magreb . C\' è chi ricorda che ciclicamente si ripetono le stesse scene , c\' è chi invece vede l\' inizio di un periodo sempre più negativo sul problema alimentare mondiale . I paesi produttori ed esportatori di grano , come la Francia , stanno assumendo un atteggiamento sempre più prudente nel promettere la consegna di partite di cereali , riaffermando la necessità di ricostruire gli stock . I prezzi nel frattempo da 100 euro la tonnellata sono già saliti al doppio e c\' è anche chi prospetta la possibilità di ulteriori rialzi . Ma sulle cause del fenomeno c\' è ancora incertezza .

# REBEL SECTION

In [ ]:
!pip install transformers wikidata requests rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
from wikidata.client import Client
import requests
import rdflib

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", src_lang="it_IT", tgt_lang="tp_XX")

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/mrebel-large")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

In [ ]:
def extract_relations_from_model_output(text):
    relations = []
    relation = ''
    subject, relation, object_, object_type, subject_type = '','','','',''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__en__", "").split():
        if token == "<triplet>"  or token == "<relation>":
            current = 't'
            if relation != '':
                relations.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                relation = ''
            subject = ''
        elif token.startswith("<") and token.endswith(">"):
            if current == 't' or current == 'o':
                current = 's'
                if relation != '':
                    relations.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                object_ = ''
                subject_type = token[1:-1]
            else:
                current = 'o'
                object_type = token[1:-1]
                relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
        relations.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
    return relations

In [ ]:
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3,
        "forced_bos_token_id": None,
    }
    generated_tokens = model.generate(
        inputs["input_ids"].to(model.device),
        attention_mask=inputs["attention_mask"].to(model.device),
        decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // 3
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [ ]:
import requests
import ast

class SemanticTripleConverter:
    def __init__(self, entities, relations):
        self.entities = entities
        self.relations = relations
        self.url = "https://lov.linkeddata.es/dataset/lov/api/v2/term/search"
        self.ontology_predicates = {'is': 'http://www.w3.org/2002/07/owl/sameAs'}
        self.RDFprefixes_names = {
            'vocabulary_prefixes': {},
            'prefixed_names': {}
        }

    def get_valid_predicate(self, predicate):
        if predicate in self.ontology_predicates:
            return self.ontology_predicates[predicate]

        external_predicate = self.fetch_data(predicate)
        if external_predicate:
            self.ontology_predicates[predicate] = external_predicate['uri']
            return external_predicate['uri']

        return predicate

    def extract_and_strip(self, data):
      try:
          # Check if 'results' key exists and has at least one item
          if 'results' in data and len(data['results']) > 0:
              result = data['results'][0]

              # Initialize the variables with default values
              prefixed_name = None
              vocabulary_prefix = None
              uri = None

              if 'prefixedName' in result and len(result['prefixedName']) > 0:
                  prefixed_name = result['prefixedName'][0].strip()

              if 'vocabulary.prefix' in result and len(result['vocabulary.prefix']) > 0:
                  vocabulary_prefix = result['vocabulary.prefix'][0].strip()

              if 'uri' in result and len(result['uri']) > 0:
                  uri = result['uri'][0].strip()

              # Ensure all required data is found
              if prefixed_name and vocabulary_prefix and uri:
                  # Store the prefixes and names
                  if vocabulary_prefix not in self.RDFprefixes_names['vocabulary_prefixes']:
                      self.RDFprefixes_names['vocabulary_prefixes'][vocabulary_prefix] = []
                  if uri not in self.RDFprefixes_names['vocabulary_prefixes'][vocabulary_prefix]:
                      self.RDFprefixes_names['vocabulary_prefixes'][vocabulary_prefix].append(uri)

                  if prefixed_name not in self.RDFprefixes_names['prefixed_names']:
                      self.RDFprefixes_names['prefixed_names'][prefixed_name] = uri

                  return {
                      'prefixedName': prefixed_name,
                      'vocabulary.prefix': vocabulary_prefix,
                      'uri': uri
                      }
              else:
                  print("Required data fields are missing in the result.")
                  return None
          else:
              print("No results found in data.")
              return None
      except (KeyError, IndexError, TypeError) as e:
          # Handle any other potential errors and print the data that caused the error
          print(f"Error processing data: {data}, error: {e}")
          return None


    def fetch_data(self, query):
        # Define the parameters for the request using the query variable
        params = {
            'q': query,
            'type': 'property',
            'page_size': 1
        }

        # Make the GET request to the API
        response = requests.get(self.url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            # Extract and strip the required information
            extracted_data = self.extract_and_strip(data)
            return extracted_data
        else:
            print(f"Error: {response.status_code}")
            return None

    def get_rdf_prefixes(self):
        # Implement the actual logic for RDF prefix extraction
        return self.RDFprefixes_names

    def get_vocabulary_prefixes(self):
        return self.RDFprefixes_names['vocabulary_prefixes']

    def get_prefixed_names(self):
        return self.RDFprefixes_names['prefixed_names']


class KB:
    def __init__(self):
        self.entities = {}
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"] if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def get_wikidata_data(self, candidate_entity, language='it'):
        from wikidata.client import Client
        client = Client()
        search_url = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbsearchentities',
            'format': 'json',
            'language': language,
            'search': candidate_entity,
            'limit': 1
        }

        try:
            response = requests.get(search_url, params=params)
            search_results = response.json()

            if not search_results.get('search'):
                return None

            entity_id = search_results['search'][0]['id']
            wikidata_entity = client.get(entity_id, load=True)
            title = wikidata_entity.label.get(language, wikidata_entity.label.get('en'))

            entity_data = {
                "title": title,
                "url": f"https://www.wikidata.org/wiki/{entity_id}",
            }
            return entity_data
        except Exception as e:
            print(f"Error fetching data: {e}")
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k: v for k, v in e.items() if k != "title"}

    def add_relation(self, r):
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikidata_data(ent) for ent in candidate_entities]

        for i, e in enumerate(entities):
            if e is None:
                entities[i] = {"title": candidate_entities[i]}
            else:
                self.add_entity(e)

        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def ensure_string(self, value):
        # Check if value is a string representation of a list
        if isinstance(value, str) and value.startswith("[") and value.endswith("]"):
            try:
                # Safely parse the string as a list using ast.literal_eval
                parsed_value = ast.literal_eval(value)
                # Check if it's a list and has one element, then return the element as a string
                if isinstance(parsed_value, list) and len(parsed_value) == 1:
                    return str(parsed_value[0])
            except (ValueError, SyntaxError):
                # Handle cases where the parsing fails
                pass
        # Default case, convert the value to string
        return str(value)

    def convert_triples(self):
        converter = SemanticTripleConverter(self.entities, self.relations)
        triples = []

        for entity in self.entities.keys():
            triple = {
                'subject': entity,
                'predicate': converter.get_valid_predicate('is'),
                'object': self.entities[entity]['url']
            }
            triples.append(triple)

        for relation in self.relations:
            value = converter.get_valid_predicate(relation['type'])
            triple = {
                'subject': relation['head'],
                'predicate': self.ensure_string(value),
                'object': relation['tail']
            }
            triples.append(triple)

        return triples

    def rdf_extraction(self):
        converter = SemanticTripleConverter(self.entities, self.relations)
        for entity in self.entities.values():
            converter.get_valid_predicate('is')
        for relation in self.relations:
            converter.get_valid_predicate(relation['type'])
        return converter.get_rdf_prefixes()

# Tryout on one article

In [ ]:
# This first tryout demonstrates that after a few triples the model confuses itself when working on a single article

first_try = dataset_list[1]
text = first_try['article']
kb = from_text_to_kb(text)
final_triples = kb.convert_triples()
final_triples

Error processing data: {'total_results': 0, 'page': 1, 'page_size': '1', 'queryString': 'Francia', 'filters': {'type': 'property'}, 'aggregations': {'types': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': []}, 'vocabs': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': []}, 'tags': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': []}}, 'results': []}
Error processing data: {'total_results': 0, 'page': 1, 'page_size': '1', 'queryString': 'cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali cereali 

[{'subject': 'rivolta del pane',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q5465517'},
 {'subject': 'Maghreb',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q28227'},
 {'subject': 'Tunisia',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q948'},
 {'subject': 'Marocco',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q1028'},
 {'subject': 'Francia',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q142'},
 {'subject': 'cereale',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q12117'},
 {'subject': 'grano',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q15645384'},
 {'subject': 'Amartya Sen',
  'predicate': 'http://www.w3.org/2002/07

Tryout on a group of sentences, instead of the full article

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
first_try = dataset_list[1]
text = first_try['article']

In [ ]:
sentences = sent_tokenize(text, language='italian')

grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
print(type(grouped_sentences))
grouped_sentences[0]

<class 'list'>


'50 morti in Tunisia , scontri in Algeria e tensioni in Marocco , aumenti di prezzo per grano , olio , zucchero , oltre che del petrolio . La sommossa ricorda quella dell\' assalto al forno delle grucce , di manzoniana memoria . È di nuovo la " rivolta del pane " , in tutto il Magreb .'

In [ ]:
first_try = dataset_list[1]
text = first_try['article']
sentences = sent_tokenize(text, language='italian')
grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
example = grouped_sentences[0]
kb = from_text_to_kb(example)
final_triples = kb.convert_triples()
final_triples

[{'subject': 'rivolta del pane',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q5465517'},
 {'subject': 'Maghreb',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q28227'},
 {'subject': 'Tunisia',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q948'},
 {'subject': 'Marocco',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q1028'},
 {'subject': 'rivolta del pane',
  'predicate': 'http://vocab.resc.info/communication#incidentLocation',
  'object': 'Maghreb'},
 {'subject': 'Tunisia',
  'predicate': 'http://purl.org/dc/elements/1.1/relation',
  'object': 'Marocco'},
 {'subject': 'Marocco',
  'predicate': 'http://purl.org/dc/elements/1.1/relation',
  'object': 'Tunisia'},
 {'subject': '50 morti in Tunisia',
  'predicate': 'http://www.europeana.eu/schemas/edm/country',
  'object': 'Tunisia'}]

In [ ]:
rdf_prefixes = kb.rdf_extraction()
rdf_prefixes

# una lista per vocabulary prefixes perché così quando posso accedere al prefix da uri diversi collezionati da tutto l'articolo
# un dict per prefixed names perché così quando devo creare l'rdf posso usare direttamente la proprietà corretta che corrisponde all'uri che ho nella tripla

# perché non creo direttamente l'rdf?
# perché triples formato più versatil (è già json pronto per altri tipi di grafi)
# tra l'altro facciamo l'esperimento di caricare il formato json-ld in apache jena
# invece di rdf

{'vocabulary_prefixes': {'swpo': ['http://sw-portal.deri.org/ontologies/swportal#hasLocation'],
  'dce': ['http://purl.org/dc/elements/1.1/relation'],
  'edm': ['http://www.europeana.eu/schemas/edm/country']},
 'prefixed_names': {'swpo:hasLocation': 'http://sw-portal.deri.org/ontologies/swportal#hasLocation',
  'dce:relation': 'http://purl.org/dc/elements/1.1/relation',
  'edm:country': 'http://www.europeana.eu/schemas/edm/country'}}

In [ ]:
print(type(rdf_prefixes))

<class 'dict'>


Tryout on a full article with the method that groups sentences to ease the process

In [ ]:
import json

In [ ]:
triples = []

article_dict = dataset_list[1]
text = article_dict['article']
sentences = sent_tokenize(text, language='italian')
grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
for j in range(len(grouped_sentences)):
    little_group = grouped_sentences[j]
    kb = from_text_to_kb(little_group)
    article_triples = kb.convert_triples()
    triples.append(article_triples)

triples

[[{'subject': 'rivolta del pane',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q5465517'},
  {'subject': 'Maghreb',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q28227'},
  {'subject': 'Tunisia',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q948'},
  {'subject': 'Marocco',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q1028'},
  {'subject': 'rivolta del pane',
   'predicate': 'http://vocab.resc.info/communication#incidentLocation',
   'object': 'Maghreb'},
  {'subject': 'Tunisia',
   'predicate': 'http://purl.org/dc/elements/1.1/relation',
   'object': 'Marocco'},
  {'subject': 'Marocco',
   'predicate': 'http://purl.org/dc/elements/1.1/relation',
   'object': 'Tunisia'},
  {'subject': '50 morti in Tunisia',
   'predicate': 'http://www.europeana.eu/schemas/edm/country',


# Create triples for two example articles

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def create_triples_example(dataset_example):
    triples = []
    prefixes_dict = {'vocabulary_prefixes': {}, 'prefixed_names': {}}

    article_dict = dataset_example
    text = article_dict['article']
    sentences = sent_tokenize(text, language='italian')
    grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]

    for j in range(len(grouped_sentences)):
        little_group = grouped_sentences[j]
        kb = from_text_to_kb(little_group)

        article_triples = kb.convert_triples()
        triples.append(article_triples)

        rdf_prefixes = kb.rdf_extraction()

        # Update nested dictionaries manually
        for key in rdf_prefixes:
            if key in prefixes_dict and isinstance(prefixes_dict[key], dict):
                prefixes_dict[key].update(rdf_prefixes[key])
            else:
                prefixes_dict[key] = rdf_prefixes[key]

    return triples, prefixes_dict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
example_triples1, example_RDFprefixes1 = create_triples_example(dataset_list[1])
example_triples2, example_RDFprefixes2 = create_triples_example(dataset_list[5])

In [ ]:
example_RDFprefixes1

{'vocabulary_prefixes': {'swpo': ['http://sw-portal.deri.org/ontologies/swportal#hasLocation'],
  'dce': ['http://purl.org/dc/elements/1.1/relation'],
  'edm': ['http://www.europeana.eu/schemas/edm/country'],
  'lemon': ['http://lemon-model.net/lemon#isA'],
  'pmovn': ['http://premon.fbk.eu/ontology/vn#subclassOf'],
  'dbpedia-owl': ['http://dbpedia.org/ontology/award'],
  'schema': ['http://schema.org/winner'],
  'rdarel2': ['http://metadataregistry.org/uri/schema/RDARelationshipsGR2/employer'],
  'vra': ['http://simile.mit.edu/2003/10/ontologies/vraCore3#movement'],
  'gvp': ['http://vocab.getty.edu/ontology#tgn3317_member_of'],
  'being': ['http://contextus.net/ontology/ontomedia/ext/common/being#place_of_citizenship']},
 'prefixed_names': {'swpo:hasLocation': 'http://sw-portal.deri.org/ontologies/swportal#hasLocation',
  'dce:relation': 'http://purl.org/dc/elements/1.1/relation',
  'edm:country': 'http://www.europeana.eu/schemas/edm/country',
  'lemon:isA': 'http://lemon-model.net/

In [ ]:
example_RDFprefixes2

{'vocabulary_prefixes': {'dbpedia-owl': ['http://dbpedia.org/ontology/subsidiary'],
  'lemon': ['http://lemon-model.net/lemon#isA'],
  'gvp': ['http://vocab.getty.edu/ontology#tgn3612_administrative_entity_for'],
  'locah': ['http://data.archiveshub.ac.uk/def/item'],
  'voag': ['http://voag.linkedmodel.org/voag#ownedBy'],
  'ngeo': ['http://geovocab.org/geometry#partOf']},
 'prefixed_names': {'dbpedia-owl:subsidiary': 'http://dbpedia.org/ontology/subsidiary',
  'dbpedia-owl:different': 'http://dbpedia.org/ontology/different',
  'lemon:isA': 'http://lemon-model.net/lemon#isA',
  'gvp:tgn3612_administrative_entity_for': 'http://vocab.getty.edu/ontology#tgn3612_administrative_entity_for',
  'locah:item': 'http://data.archiveshub.ac.uk/def/item',
  'voag:ownedBy': 'http://voag.linkedmodel.org/voag#ownedBy',
  'ngeo:partOf': 'http://geovocab.org/geometry#partOf'}}

Save the triples to an external json file

In [ ]:
example_triples = example_triples1 + example_triples2

In [ ]:
import json

file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_API_triples_nolabel.json'

# Write to the JSON file in Google Drive
with open(file_path, 'w') as file:
    json.dump(example_triples, file, indent=4)

Save the RDF prefixes to an external json file

In [ ]:
example_RDFprefixes1['vocabulary_prefixes']

{'swpo': ['http://sw-portal.deri.org/ontologies/swportal#hasLocation'],
 'dce': ['http://purl.org/dc/elements/1.1/relation'],
 'edm': ['http://www.europeana.eu/schemas/edm/country'],
 'lemon': ['http://lemon-model.net/lemon#isA'],
 'pmovn': ['http://premon.fbk.eu/ontology/vn#subclassOf'],
 'dbpedia-owl': ['http://dbpedia.org/ontology/award'],
 'schema': ['http://schema.org/winner'],
 'rdarel2': ['http://metadataregistry.org/uri/schema/RDARelationshipsGR2/employer'],
 'vra': ['http://simile.mit.edu/2003/10/ontologies/vraCore3#movement'],
 'gvp': ['http://vocab.getty.edu/ontology#tgn3317_member_of'],
 'being': ['http://contextus.net/ontology/ontomedia/ext/common/being#place_of_citizenship']}

In [ ]:
general_prefixes = {'vocabulary_prefixes': {}, 'prefixed_names': {}}

general_prefixes['vocabulary_prefixes'] = {**example_RDFprefixes1['vocabulary_prefixes'], **example_RDFprefixes2['vocabulary_prefixes']}

general_prefixes['prefixed_names'] = {**example_RDFprefixes1['prefixed_names'], **example_RDFprefixes2['prefixed_names']}

general_prefixes

{'vocabulary_prefixes': {'swpo': ['http://sw-portal.deri.org/ontologies/swportal#hasLocation'],
  'dce': ['http://purl.org/dc/elements/1.1/relation'],
  'edm': ['http://www.europeana.eu/schemas/edm/country'],
  'lemon': ['http://lemon-model.net/lemon#isA'],
  'pmovn': ['http://premon.fbk.eu/ontology/vn#subclassOf'],
  'dbpedia-owl': ['http://dbpedia.org/ontology/subsidiary'],
  'schema': ['http://schema.org/winner'],
  'rdarel2': ['http://metadataregistry.org/uri/schema/RDARelationshipsGR2/employer'],
  'vra': ['http://simile.mit.edu/2003/10/ontologies/vraCore3#movement'],
  'gvp': ['http://vocab.getty.edu/ontology#tgn3612_administrative_entity_for'],
  'being': ['http://contextus.net/ontology/ontomedia/ext/common/being#place_of_citizenship'],
  'locah': ['http://data.archiveshub.ac.uk/def/item'],
  'voag': ['http://voag.linkedmodel.org/voag#ownedBy'],
  'ngeo': ['http://geovocab.org/geometry#partOf']},
 'prefixed_names': {'swpo:hasLocation': 'http://sw-portal.deri.org/ontologies/swpor

In [ ]:
example_rdf_prefixes =  '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_rdf_prefixes.json'

with open(example_rdf_prefixes, 'w') as json_file:
    json.dump(general_prefixes, json_file, indent=4)

Creating triples for the whole dataset - TO BE RUN (BUT IT TAKES TOO MUCH TIME!)

In [ ]:
def generate_triples_labelled(dataset_list):

    triples = []

    for i in range(len(dataset_list)):
        article_dict = dataset_list[i]
        text = article_dict['article']
        sentences = sent_tokenize(text, language='italian')
        grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
        for j in range(len(grouped_sentences)):
            little_group = grouped_sentences[j]
            kb = from_text_to_kb(little_group)
            article_triples = kb.convertJSON()
            triples.append(article_triples)

    return triples

In [ ]:
triples = generate_triples_labelled(dataset_list)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-db8ccafb56bb>", line 1, in <cell line: 1>
    triples = generate_triples_labelled(dataset_list)
  File "<ipython-input-23-58f64399197b>", line 12, in generate_triples_labelled
    kb = from_text_to_kb(little_group)
  File "<ipython-input-12-224df637e982>", line 61, in from_text_to_kb
    kb.add_relation(relation)
  File "<ipython-input-13-e98ff237c9ee>", line 58, in add_relation
    entities = [self.get_wikidata_data(ent) for ent in candidate_entities]
  File "<ipython-input-13-e98ff237c9ee>", line 58, in <listcomp>
    entities = [self.get_wikidata_data(ent) for ent in candidate_entities]
  File "<ipython-input-13-e98ff237c9ee>", line 41, in get_wikidata_data
    wikidata_entity = client.get(entity_id, load=True)
  File "/usr/local/lib/python3.10/dist-packages/wikidat

TypeError: object of type 'NoneType' has no len()